# Procedure to handle IP address

## Init MISP

In [ ]:
import urllib3
from pymisp import PyMISP, MISPEvent
import sys
sys.path.insert(0, "/home/user/cti-operational-procedure/vault/")
from keys import *

if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

misp = PyMISP(misp_url, misp_key, misp_verifycert)

import requests

misp_headers = {
            "Authorization": misp_key,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

import tabulate

print("I will use the server {}".format(misp_url))

## Analyst: set IP address

In [ ]:
investigate_ip = "200.159.87.196"

## Lookup in the cache

In [ ]:
misp_cache_url = "{}/feeds/searchCaches/value:{}".format(misp_url, investigate_ip)
cache_results = requests.get(misp_cache_url, headers=misp_headers, verify=misp_verifycert)
direct_urls = []
for el in cache_results.json():
    direct_urls.append(el["Feed"]["direct_urls"])

## Fetch the cached events

In [ ]:
cache_table = []
for el in direct_urls[0]:
    misp_preview_cache_url = el["url"]
    cached_event = requests.get(misp_preview_cache_url, headers=misp_headers, verify=misp_verifycert)
    
    cached_event_title = cached_event.json()["Event"]["info"]
    cached_event_data = cached_event.json()["Event"]["date"]
    cache_table.append([cached_event_title, cached_event_data])


In [ ]:
print(tabulate.tabulate(cache_table, headers=["MISP Event title", "Date"]))

In [ ]:
import pypdns
x = pypdns.PyPDNS(basic_auth=(pdns_user,pdns_password))
pdns_results = x.query(investigate_ip)
pdns_results_table = []
if len(pdns_results) > 0:
    for el in pdns_results:
        pdns_results_table.append([el["rdata"], el["rrname"], el["rrtype"]])

print(tabulate.tabulate(pdns_results_table, headers=["Data", "Name", "Type"]))